In [26]:
from os import path,getcwd
import pandas as pd
import numpy as np

from scipy.stats import shapiro
from scipy.stats import pearsonr
from fitter import Fitter


from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_white"

- 1. Verificar se as amostras seguem uma Distribuição Normal, Se pertencem a mesma distribuição e se são Independentes. Plotar os gráficos.

- 2. DESAFIO: Inserir as estatísticas de Média e Desvio Padrão no gráfico


In [27]:
path.root = path.join(getcwd(),'..')
path.data = path.join(path.root,'data')
path.output = path.join(path.data,'output')
path.input = path.join(path.data,'input')

In [28]:
lista_estados = ['Chicago', 'HarrisburgScranton', 'Pittsburgh','Boise', 'LosAngeles', 'LasVegas', 'Atlanta','DallasFtWorth', 'MiamiFtLauderdale','StLouis','Syracuse', 'Sacramento', 'Boston', 'Charlotte','Spokane', 'Albany', 'Houston', 'SanFrancisco','RaleighGreensboro', 'BuffaloRochester','GrandRapids', 'Denver', 'SanDiego','BaltimoreWashington', 'Roanoke', 'RichmondNorfolk', 'Louisville', 'Seattle','CincinnatiDayton', 'NewYork', 'NewOrleansMobile', 'Indianapolis','Jacksonville', 'Columbus', 'Detroit', 'Philadelphia','PhoenixTucson', 'Nashville', 'Portland', 'HartfordSpringfield','Tampa', 'Orlando',]
lista_regioes = ['Southeast', 'Plains',  'Midsouth', 'NorthernNewEngland', 'Northeast', 'SouthCentral', 'GreatLakes', 'West','WestTexNewMexico',]

In [29]:
path_read = path.join(path.input,'avocado.csv')

names = ['date','average_price','total_volume','PLU_4046','PLU_4225','PLU_4770','total_bags','small_bags','large_bags','XLarge_bags','type','year','region']

df = pd.read_csv(
        path_read,
        index_col=0,
        names=names,
        header=0,
        parse_dates=['date']
)


df = df.sort_values(
            by='date',
            ignore_index=True
)

df

,date,average_price,total_volume,PLU_4046,PLU_4225,PLU_4770,total_bags,small_bags,large_bags,XLarge_bags,type,year,region
0,2015-01-04,1.75,27365.89,9307.34,3844.81,615.28,13598.46,13061.10,537.36,0.00,organic,2015,Southeast
1,2015-01-04,1.49,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.00,organic,2015,Chicago
2,2015-01-04,1.68,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.00,organic,2015,HarrisburgScranton
3,2015-01-04,1.52,54956.80,3013.04,35456.88,1561.70,14925.18,11264.80,3660.38,0.00,conventional,2015,Pittsburgh
4,2015-01-04,1.64,1505.12,1.27,1129.50,0.00,374.35,186.67,187.68,0.00,organic,2015,Boise
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,2018-03-25,1.36,908202.13,142681.06,463136.28,174975.75,127409.04,103579.41,22467.04,1362.59,conventional,2018,Chicago
18245,2018-03-25,0.70,9010588.32,3999735.71,966589.50,30130.82,4014132.29,3398569.92,546409.74,69152.63,conventional,2018,SouthCentral
18246,2018-03-25,1.42,163496.70,29253.30,5080.04,0.00,129163.36,109052.26,20111.10,0.00,organic,2018,SouthCentral
18247,2018-03-25,1.70,190257.38,29644.09,70982.10,0.00,89631.19,89424.11,207.08,0.00,organic,2018,California


In [38]:
_df_1 = df.sample(frac=0.3)
_df_2 = df.sample(frac=0.3)

mask_estados = (df.type == 'conventional') & df.region.isin(lista_estados)
_mask_estados_1 = (_df_1.type == 'conventional') & _df_1.region.isin(lista_estados)
_mask_estados_2 = (_df_2.type == 'conventional') & _df_2.region.isin(lista_estados)

average_price_estados   =  df.loc[mask_estados, ['date','average_price']].groupby('date')['average_price'].mean()
_average_price_estados_1  = _df_1.loc[_mask_estados_1, ['date','average_price']].groupby('date')['average_price'].mean()
_average_price_estados_2  = _df_2.loc[_mask_estados_2, ['date','average_price']].groupby('date')['average_price'].mean()

data_1 = _average_price_estados_1.values
data_2 = _average_price_estados_2.values

fig = ff.create_distplot(
    [data_1,data_2],
    group_labels=['amostra 1','amostra 2'],
    bin_size=.1,
    show_rug=True,
    colors = px.colors.qualitative.Pastel
)
stat, p = shapiro(data_1)

print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Provavelmente uma Distribuição Normal')
else:
	print('Provavelmente NÃO é uma Distribuição Normal')

fig.show()

stat=0.931, p=0.000
Provavelmente NÃO é uma Distribuição Normal


In [39]:
stat, p = pearsonr(data_1, data_2)

print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Provavelmente Independente')
else:
	print('Provavelmente Dependente')

stat=0.905, p=0.000
Provavelmente Dependente


In [40]:
f = Fitter(data_1,)
f.fit()

fig = px.bar(f.y, labels=['amostra'])

counter = 0

list_colors = px.colors.qualitative.Pastel + px.colors.qualitative.Pastel2 + px.colors.qualitative.Set3 + px.colors.qualitative.Set2 + px.colors.qualitative.Set1 + px.colors.qualitative.Light24 + px.colors.qualitative.Dark24  + px.colors.qualitative.Light24 + px.colors.qualitative.Dark24  + px.colors.qualitative.Light24 + px.colors.qualitative.Dark24  + px.colors.qualitative.Light24 + px.colors.qualitative.Dark24  + px.colors.qualitative.Light24 + px.colors.qualitative.Dark24

for name in f.fitted_pdf:
    fig.add_trace(
        go.Scatter(
            y=f.fitted_pdf[name],
            name=name,
            marker=dict(color = list_colors[counter])
        )    
    )
    counter += 1
fig.show()

In [41]:
name = list(f.get_best().keys())[0]
fig = px.bar(f.y, labels=['amostra'])

fig.add_trace(
    go.Scatter(
        y=f.fitted_pdf[name],
        name=name
    )
)
fig.show()

C:\Users\jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\quadpack.py:879: IntegrationWarning:

The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.



# Parte 4

In [10]:
_df_1 = df.sample(frac=0.3)
_df_2 = df.sample(frac=0.3)

mask_estados = (df.type == 'conventional') & df.region.isin(lista_estados)
_mask_estados_1 = (_df_1.type == 'conventional') & _df_1.region.isin(lista_estados)
_mask_estados_2 = (_df_2.type == 'conventional') & _df_2.region.isin(lista_estados)

average_price_estados   =  df.loc[mask_estados, ['date','average_price']].groupby('date')['average_price'].mean()
_average_price_estados_1  = _df_1.loc[_mask_estados_1, ['date','average_price']].groupby('date')['average_price'].mean()
_average_price_estados_2  = _df_2.loc[_mask_estados_2, ['date','average_price']].groupby('date')['average_price'].mean()

data_1 = _average_price_estados_1.values
data_2 = _average_price_estados_2.values

fig = ff.create_distplot(
    [data_1,data_2],
    group_labels=['a','b'],
    bin_size=.1,
    show_rug=True,
    colors = px.colors.qualitative.Pastel
)
stat, p = shapiro(data_1)

print('stat=%.3f, p=%.3f' % (stat, p))

if p > 0.05:
	print('Provavelmente uma Distribuição Normal')
else:
	print('Provavelmente NÃO é uma Distribuição Normal')

fig.show()

C:\Users\jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\quadpack.py:879: IntegrationWarning:

The integral is probably divergent, or slowly convergent.



stat=0.951, p=0.000
Provavelmente NÃO é uma Distribuição Normal


In [11]:
pd.DataFrame(([dict(teste=1),dict(p=1)]))

,teste,p
0,1.0,NaN
1,NaN,1.0


In [12]:
resultado = pd.DataFrame()

for i in range(100):
    print(i)
    
    _df_1 = df.sample(frac=0.3)
    _df_2 = df.sample(frac=0.3)

    mask_estados = (df.type == 'conventional') & df.region.isin(lista_estados)
    _mask_estados_1 = (_df_1.type == 'conventional') & _df_1.region.isin(lista_estados)
    _mask_estados_2 = (_df_2.type == 'conventional') & _df_2.region.isin(lista_estados)

    average_price_estados   =  df.loc[mask_estados, ['date','average_price']].groupby('date')['average_price'].mean()
    _average_price_estados_1  = _df_1.loc[_mask_estados_1, ['date','average_price']].groupby('date')['average_price'].mean()
    _average_price_estados_2  = _df_2.loc[_mask_estados_2, ['date','average_price']].groupby('date')['average_price'].mean()

    data_1 = _average_price_estados_1.values
    data_2 = _average_price_estados_2.values

    _, p_shapiro = shapiro(data_1)
    _, p_pearsonr = pearsonr(data_1, data_2)


    #print('stat=%.3f, p=%.3f' % (stat, p))
    resultado = resultado.append([dict(teste=i,shapiro=p_shapiro,pearsonr=p_pearsonr)])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58


C:\Users\jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\stats\_continuous_distns.py:4837: IntegrationWarning:

The integral is probably divergent, or slowly convergent.



59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [13]:
(resultado.shapiro > 0.05).sum()

0

In [14]:
(resultado.pearsonr > 0.05).sum()

0

In [17]:
resultado = pd.DataFrame()

for i in range(15):
    df_1 = df.sample(frac=0.3)

    _mask_estados_1 = (_df_1.type == 'conventional') & _df_1.region.isin(lista_estados)
    
    _average_price_estados_1  = _df_1.loc[_mask_estados_1, ['date','average_price']].groupby('date')['average_price'].mean()
    
    data_1 = _average_price_estados_1.values

    f = Fitter(data_1)
    f.fit()
    name = list(f.get_best().keys())[0]
    resultado = resultado.append([dict(teste=i,best=name)])

resultado

C:\Users\jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\quadpack.py:879: IntegrationWarning:

The integral is probably divergent, or slowly convergent.

C:\Users\jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\quadpack.py:879: IntegrationWarning:

The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.

C:\Users\jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Py

In [45]:
dist = average_price_estados.values

f = Fitter(dist,
        distributions=[
            'laplace_asymmetric',
            'dweibull',
            'exponnorm',
            'burr12',
            'loglaplace',
            'johnsonsu',
            'alpha'
        ]
        )
f.fit()

fig = px.bar(f.y, labels=['completo'])

counter = 0

list_colors = px.colors.qualitative.Pastel + px.colors.qualitative.Pastel2 + px.colors.qualitative.Set3 + px.colors.qualitative.Set2 + px.colors.qualitative.Set1 + px.colors.qualitative.Light24 + px.colors.qualitative.Dark24  + px.colors.qualitative.Light24 + px.colors.qualitative.Dark24  + px.colors.qualitative.Light24 + px.colors.qualitative.Dark24  + px.colors.qualitative.Light24 + px.colors.qualitative.Dark24  + px.colors.qualitative.Light24 + px.colors.qualitative.Dark24

for name in f.fitted_pdf:
    fig.add_trace(
        go.Scatter(
            y=f.fitted_pdf[name],
            name=name,
            marker=dict(color = list_colors[counter])
        )    
    )
    counter += 1
fig.show()

C:\Users\jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\stats\_continuous_distns.py:4837: IntegrationWarning:

The integral is probably divergent, or slowly convergent.

C:\Users\jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\stats\_continuous_distns.py:4837: IntegrationWarning:

The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.

